In [1]:
import json
from collections import Counter
import regex as re
import os
# from transformers import AutoTokenizer #4.25.1 torch 1.7.1
import pyphen
import pandas as pd
from tokenizers import Tokenizer
from sklearn.preprocessing import MinMaxScaler
import random
from tokenizers import ByteLevelBPETokenizer
from tokenizers.pre_tokenizers import ByteLevel

ModuleNotFoundError: No module named 'pyphen'

In [ ]:
folder_to_process='strict_model_ordered_data_reverse'
path='./SlovakBabyLM/Curricullum_learning/'

### Train tokenizer

In [ ]:
import os
import json

load_json_path=f'{path}{folder_to_process}'
save_path=f'{path}tok_bpe/{folder_to_process}_BPE/text_tokenizer.txt'
tokenizer_dir = f"{path}tok_bpe/{folder_to_process}_BPE"
# if you work with whole_data you need to put it into path

os.makedirs(tokenizer_dir, exist_ok=True)


def define_txt_file(load_path,save_path):
    text=''
    for subdir, dirs, files in os.walk(load_path):
        for file in files: 
            file_path = os.path.join(subdir, file)
            with open(file_path, "r", encoding="utf8") as f:
                if not "_processed" in file_path and not "_reordered" in file_path:
                    # print(f'\n Start Process: {file_path}')
                    data = json.load(f)
                    for i in data:
                        if isinstance(i['page'],list):
                            i['page']=' '.join(i['page'])
                        text+=i['page']
    with open(save_path, "w", encoding="utf-8") as f:
        f.write(text)


tokenizer = ByteLevelBPETokenizer()
tokenizer.pre_tokenizer = ByteLevel()


text= define_txt_file(load_json_path,save_path)

tokenizer.train(
    [save_path],
    vocab_size=60000, 
    min_frequency=2, 
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    show_progress=True
)

tokenizer_json_path = os.path.join(tokenizer_dir, "tokenizer.json")
tokenizer.save(tokenizer_json_path)


In [ ]:
tokenizer = Tokenizer.from_file(f"{path}{folder_to_process}_BPE/tokenizer.json")

# FREQUENCY

Compute word frequency and bi-gram frequency 

In [40]:
def generate_bigrams(words):
    return [(words[i], words[i + 1]) for i in range(len(words) - 1)]


def compute_frequency_words(file_path,full_word_freq,full_bi_freq,full_tokens_freq):
    try:
        if not "_processed" in file_path and not "_reordered" in file_path:
            with open(file_path, "r", encoding="utf8") as f:
                print(f'Processed freq:{file_path}')
                data = json.load(f)
                for source in data:
                    tokens_source=tokenizer.encode(source['page']).tokens
                    #tokens freq
                    tokens_freq = Counter(tokens_source)
                    full_tokens_freq.update(tokens_freq)                          
                    #source to words
                    words = source['page'].split()
                    words=[re.sub(r'[^\p{L}\p{N} ]+', '', item) for item in words]
                    bi_gram=generate_bigrams(words)
                    #bi_gram
                    bi_freq = Counter(bi_gram)
                    full_bi_freq.update(bi_freq)
                    #individual_words
                    word_freq = Counter(words)
                    full_word_freq.update(word_freq)
            # freq_dict[file_path]={'token_f':full_tokens_freq,'bi_f':full_bi_freq,'word_f':full_word_freq}
    except (json.JSONDecodeError, IOError) as e:
        print(f"Error reading {file_path}: {e}")
    return full_word_freq,full_bi_freq,full_tokens_freq



def bi_gram_freq(source,bi_freq):
    source_all_bi_gram=[]
    bi_gram=generate_bigrams(source)
    for bi in bi_gram:
        source_all_bi_gram.append(bi_freq[bi])
    return source_all_bi_gram

def tok_freq(s,tokens_freq):
    source_all_tok=[]
    tokens_source=tokenizer.encode(s['page']).tokens
    for token in tokens_source:
        source_all_tok.append(tokens_freq[token])
    return source_all_tok

Folders

In [ ]:
full_word_freq = Counter()
full_bi_freq = Counter()
full_tokens_freq = Counter()
for subdir, dirs, files in os.walk(f'{path}{folder_to_process}'):
    for dir in dirs: 
        folder_path = os.path.join(subdir, dir)
        if not '_reordered' in folder_path and not '_processed' in folder_path:
            for files in os.listdir(folder_path):
                file_path = os.path.join(folder_path, files)
                full_word_freq,full_bi_freq,full_tokens_freq=compute_frequency_words(file_path,full_word_freq,full_bi_freq,full_tokens_freq)

In [ ]:
full_word_freq = Counter()
full_bi_freq = Counter()
full_tokens_freq = Counter()
for subdir, dirs, files in os.walk(f'{path}{folder_to_process}'):
    for file in files: 
        file_path = os.path.join(subdir, file)
        if not '_reordered' in file_path and not '_processed' in file_path:
            full_word_freq,full_bi_freq,full_tokens_freq=compute_frequency_words(file_path,full_word_freq,full_bi_freq,full_tokens_freq )

Processed freq:/data/lubosk/diploma_thesis/data/wiki_full/wiki_full_data.json


Full_file

In [ ]:
full_word_freq = Counter()
full_bi_freq = Counter()
full_tokens_freq = Counter()

full_word_freq,full_bi_freq,full_tokens_freq=compute_frequency_words(f'{path}{folder_to_process}_full_data/{folder_to_process}_full_data.json',full_word_freq,full_bi_freq,full_tokens_freq)

Processed freq:/data/lubosk/diploma_thesis/strict_model_ordered_data_full_data/strict_model_ordered_data_full_data.json


# Grammar

Punctuation_density

In [42]:
def punctuation_counter(text):
    pattern = r"(?<!\d)([;.?!])(?!\d)(?![;.?!])"
    punk_sum=sum(1 for _ in re.finditer(pattern,text))
    return punk_sum

In [43]:
spojky = [
    "a","že", "i","keby", "aby","aj", "ak","keď", "keďže","ako","akoby","hoci","ale", "alebo","lebo", "ani", "iba","tak", "takže", "teda", "totižto", "veď", "však", "žeby"
    "avšak", "až", "ba", "bár", "beztak", "buď","by", "či", "čím","čoby",'pričom'
    "čiže", "čo", "kým", "leda", "ledva", "len", "len čo", "totiž",
    "lenže","najprv", "nech", "než","nielen", "no", "nuž","pokiaľ", "pokým", "predsa", "preto", "pretože", "síce","sotva", "sťa", 'prípadne', 'poprípade', 'eventuálne',
]
len(spojky)

59

In [44]:
#len slová ktoré môžu byť predložky
predlozky = [
    "bez", "cez", "do", "k", "medzi", "na", "nad", "o", "od", "okrem", "po", "pod", "pre", "pred",
    "pri", "proti", "s", "skrz", "u", "v", "z", "za",
    "ponad", "popod", "popred", "poza", "popri", "pomedzi", "znad", "spred", "zmedzi",
    "spod", "spopred", "sponad", "spopod", "spoza", "spopri", "spomedzi",
    'zo','ku','voči','skrze','vo','so']

len(predlozky)

44

### Evaluation of dataset- create '_processed' files

In [45]:
def create_processed(file_path,word_f_counter,bi_f_counter,token_f_counter):
    dic=pyphen.Pyphen(lang='sk_SK')
    if not "_processed" in file_path and not "_reordered" in file_path:
        with open(file_path, "r", encoding="utf8") as f:
            print(f'\n Start Process: {file_path}')
            data = json.load(f)
        for i,s in enumerate(data):
            if not s['page']:
                continue
            punct_count=punctuation_counter(s['page'])
            source= s['page'].split()
            syl_list=[]
            w_len=[]
            freq_list=[]
            spojky_number=0
            predlozky_number=0
            for w in source: 
                w_without_dia=re.sub(r'[^\p{L}\p{N} ]+', '', w) 
                if w_without_dia in spojky:
                    spojky_number+=1
                if w_without_dia in predlozky:
                    predlozky_number+=1
                w_len.append(len(w_without_dia))
                syl_list.append(len(dic.inserted(w).split("-")))
                if w_without_dia:
                    freq_list.append(word_f_counter[w_without_dia])  

            len_functional_words=len([s for s in source if re.search(r"[A-Za-z0-9]", s)])
            try:
                source_bi_freq=bi_gram_freq(source,bi_f_counter)
                source_all_tok=tok_freq(s,token_f_counter)
                s['index']=i
                ## Frequency
                s['average_source_rarity']=-sum(freq_list)/len(freq_list)
                s['average_tok']=-sum(source_all_tok)/len(source_all_tok)
                s['average_bi_gram']=-sum(source_bi_freq)/len(source_bi_freq)  
                ## Grammar  
                s['average_word_length']=sum(w_len)/len(w_len)
                s['sylabelle/word']=sum(syl_list)/len_functional_words
                s['spojky']=spojky_number/len_functional_words
                s['predlozky']=predlozky_number/len_functional_words
                s['punctuation_density']=-punct_count/len(source)
            except Exception as e :
                print(e) 
        new_path=file_path.split(".") 
        new_path=new_path[0]+'_processed.'+new_path[1]
        with open(new_path, "w", encoding="utf8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)


Folders

In [ ]:
path=f'{path}{folder_to_process}'


for subdir, dirs, files in os.walk(path):
            for dir in dirs: 
                folder_path = os.path.join(subdir, dir)
                for files in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, files)
                    create_processed(file_path,full_word_freq,full_bi_freq,full_tokens_freq)


 Start Process: /data/lubosk/diploma_thesis/strict_model/fairytales/fairytales.json

 Start Process: /data/lubosk/diploma_thesis/strict_model/edu/edu.json

 Start Process: /data/lubosk/diploma_thesis/strict_model/childes/childes.json

 Start Process: /data/lubosk/diploma_thesis/strict_model/wiki/wiki.json

 Start Process: /data/lubosk/diploma_thesis/strict_model/subs/subs.json

 Start Process: /data/lubosk/diploma_thesis/strict_model/lit/lit.json


File

In [ ]:
path=f'{path}{folder_to_process}_full_data/{folder_to_process}_full_data.json'
create_processed(path,full_word_freq,full_bi_freq,full_tokens_freq)


 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_full_data/strict_model_ordered_data_full_data.json


### Evaluate _process: Create order of data in index form

In [ ]:
def evaluate_text(ordering_index,ordering_folders,file_path,order_by):
    # if "_processed" in file_path:
    with open(file_path, "r", encoding="utf8") as f:
        print(f'\n Start Process: {file_path}')
        data = json.load(f)
        for s in data:
            s.pop('page', None)
    df = pd.DataFrame(data)
    df.set_index('index', inplace=True)
    numeric_columns = df.select_dtypes(include=["number"]).columns
    #final order of files
    ordering_folders[file_path]=df[numeric_columns]
    #normalization
    scaler = MinMaxScaler()
    df[numeric_columns]=scaler.fit_transform(df[numeric_columns])
    # Combination
    freq_group = numeric_columns[:3]   
    gramma_group = numeric_columns[-5:]  
    df["freq_group"] = df[freq_group].sum(axis=1)
    df["gramma_group"] = df[gramma_group].sum(axis=1)
    df['sum_freq_gramma']=df["freq_group"]+df["gramma_group"]
    #sorting
    sorted_df = df.sort_values(by=order_by)
    index_order = sorted_df.index.tolist()
    #final list
    ordering_index[file_path]=index_order
    return ordering_index,ordering_folders

File

In [ ]:
path=f'{path}{folder_to_process}_full_data/{folder_to_process}_full_data_processed.json'
order_by="sum_freq_gramma"
ordering_index={}
ordering_folders={}

ordering_index,ordering_folders=evaluate_text(ordering_index,ordering_folders,path,order_by)


 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_full_data/strict_model_ordered_data_full_data_processed.json


Folders

In [ ]:
path=f'{path}{folder_to_process}'
order_by="sum_freq_gramma"
ordering_index={}
ordering_folders={}

for subdir, dirs, files in os.walk(path):
    for dir in dirs: 
        folder_path = os.path.join(subdir, dir)
        for files in os.listdir(folder_path):
            file_path = os.path.join(folder_path, files)
            ordering_index,ordering_folders=evaluate_text(ordering_index,ordering_folders,file_path,order_by)


 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/fairytales_full/fairytales_full.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/edu/edu.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/childes/childes.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/wiki_full/wiki_full.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/subs/subs.json

 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/lit/lit.json


In [5]:
for k,df in ordering_folders.items():
    ordering_folders[k]=df.mean()
df = pd.DataFrame(ordering_folders).T
df_ranked=df.apply(lambda x: x.rank(method='first', ascending=True).astype(int))
# df_ranked.index = df_ranked.index.map(lambda x: x.split('/')[-1])


In [7]:
df_ranked['Sum'] = df_ranked.sum(axis=1)
df_ranked=df_ranked.sort_values(by='Sum', ascending=True)

In [19]:
df_ranked['Sum'] = df_ranked.sum(axis=1)
df_ranked=df_ranked.sort_values(by='Sum', ascending=True)
updated_paths = [path.replace('_processed', f'_reordered_{order_by}') for path in df_ranked.index.tolist()]
updated_paths

['/data/lubosk/diploma_thesis/strict_model_ordered_data/subs/subs.json',
 '/data/lubosk/diploma_thesis/strict_model_ordered_data/childes/childes.json',
 '/data/lubosk/diploma_thesis/strict_model_ordered_data/wiki_full/wiki_full.json',
 '/data/lubosk/diploma_thesis/strict_model_ordered_data/lit/lit.json',
 '/data/lubosk/diploma_thesis/strict_model_ordered_data/fairytales_full/fairytales_full.json',
 '/data/lubosk/diploma_thesis/strict_model_ordered_data/edu/edu.json']

In [8]:
df_ranked

,average_source_rarity,average_tok,average_bi_gram,average_word_length,sylabelle/word,spojky,predlozky,punctuation_density,Sum
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/subs/subs.json,1,1,1,2,3,5,2,2,17
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/childes/childes.json,6,5,3,1,1,3,1,1,21
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/wiki_full/wiki_full.json,2,2,2,6,6,1,4,6,29
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/fairytales_full/fairytales_full.json,4,6,4,3,2,6,3,3,31
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/edu/edu.json,3,4,5,5,5,2,6,5,35
/data/lubosk/diploma_thesis/strict_model_ordered_data_reverse/lit/lit.json,5,3,6,4,4,4,5,4,35


Ordering dictionaries Create _reordered

In [53]:
import math

def order_dict_in_folders(file_path,ordering_index,order_by):    
    # if "_processed" in file_path:
    with open(file_path, "r", encoding="utf8") as f:
        print(f'\n Start Process: {file_path}')
        data = json.load(f)
    try:
        ordering_index[file_path]=[x for x in ordering_index[file_path] if not math.isnan(x)]
        if isinstance(ordering_index[file_path][0],float):
            ordering_index[file_path]=[int(x) for x in ordering_index[file_path]]
        reordered_data=[data[i] for i in ordering_index[file_path]]
        # if "_processed" in file_path:
        #     new_reorded_path=file_path.replace('_processed',f'_reordered_{order_by}')
        # else:
        file_path_split= file_path.split('.')
        new_reorded_path=file_path_split[0] + f'_reordered_{order_by}'+file_path_split[1]
        with open(new_reorded_path, "w", encoding="utf8") as f_reorded:
            json.dump(reordered_data, f_reorded, ensure_ascii=False, indent=4)
    except:
         print('e')

In [ ]:
path=f'{path}{folder_to_process}'

for subdir, dirs, files in os.walk(path):
            for dir in dirs: 
                folder_path = os.path.join(subdir, dir)
                for files in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, files)
                    order_dict_in_folders(file_path,ordering_index,order_by)


 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data/fairytales_full/fairytales_full.json
e


UnboundLocalError: local variable 'new_reorded_path' referenced before assignment

In [ ]:
path=f'{path}{folder_to_process}_full_data/{folder_to_process}_full_data_processed.json'

order_dict_in_folders(path,ordering_index,order_by)


 Start Process: /data/lubosk/diploma_thesis/strict_model_ordered_data_full_data/strict_model_ordered_data_full_data_processed.json
